In [5]:
import time 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [13]:
CHROMEDRIVER_PATH = "./chromedriver.exe"

In [16]:
def places_reviews(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)

    # url = 'https://www.tripadvisor.com/Attraction_Review-g297704-d3257737-Reviews-Braga_Street-Bandung_West_Java_Java.html'
    driver.get(url)

    time.sleep(5)

    placeName = driver.find_element("xpath", ".//h1[@class='biGQs _P fiohW eIegw']").text

    # not all place have details
    # places_details = driver.find_element('xpath', './/div[contains(@class, "biGQs _P pZUbB KxBGd")]').text

    temp = driver.find_element("xpath", ".//span[@class='biGQs _P pZUbB KxBGd']").text
    numeric_part = ''.join(filter(str.isdigit, temp))
    reviewCount = int(numeric_part)

    overallRating = driver.find_element("xpath", ".//div[@class='biGQs _P fiohW hzzSG uuBRH']").text

    # click all languages
    dropdown_button = driver.find_element("xpath", ".//button[@class='OKHdJ z Pc PQ Pp PD W _S Gn Rd _M PQFNM wSSLS']").click()
    time.sleep(1)
    all_language_button = driver.find_element("xpath", ".//span[contains(@id, 'menu-item-all')]").click()


    no_result_found = 0
    reviews = []

    # while len(reviews) < reviewCount and no_result_found != 1:
    while len(reviews) < 10 and no_result_found != 1:
        time.sleep(3)

        # Click the "expand review" link to reveal the entire review.
        try:
            driver.find_element("xpath", ".//span[@class='biGQs _P XWJSj Wb']").click()
        except ElementNotInteractableException:
            element = driver.find_element("xpath", ".//span[@class='biGQs _P XWJSj Wb']")
            driver.execute_script("arguments[0].click();", element)

        # find review in the current page
        container = driver.find_elements("xpath", "//div[contains(@data-automation, 'reviewCard')]")

        # print("panjang container: ", len(container))
        # break
        for j in range(len(container)):
            # If we have enough reviews, stop parsing
            if len(reviews) >= reviewCount:
                break
            
            # reviewer link and name
            retry_count = 0
            while retry_count < 3:
                try:
                    # grab reviewer link and name
                    reviewerLink = container[j].find_element("xpath", ".//a[@class='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS']").get_attribute("href")
                    reviewerName = container[j].find_element("xpath", ".//a[@class='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS']").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewerLink = None
                    reviewerName = None
                    break   

            # reviewer location
            # biGQs _P pZUbB osNWb
            retry_count = 0
            while retry_count < 3:
                try:
                    # grab reviewer location
                    reviewerLocation = container[j].find_element("xpath", ".//div[@class='biGQs _P pZUbB osNWb']/span").text

                    # if path .//div[@class='biGQs _P pZUbB osNWb']/span/IugUm available get the text
                    try:
                        reviewsContributions = container[j].find_element("xpath", ".//div[@class='biGQs _P pZUbB osNWb']/span/IugUm")
                        reviewsContributions = reviewerLocation.text
                    except NoSuchElementException:
                        reviewsContributions = None
                    break

                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewerLocation = None
                    break
            
            # review rating
            # UctUV d H0
            retry_count = 0
            while retry_count < 3:
                try:
                    # grab the review rating
                    # UctUV d H0
                    # reviewRating = container[j].find_element("xpath", "//*[name()='svg' and @class='UctUV d H0']").get_attribute("aria-label")
                    reviewRating = container[j].find_element("xpath", ".///svg[@class='UctUV d H0']/title").text
                    break 
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewRating = None
                    break   
            

            # date of visit
            # RpeCd
            retry_count = 0
            while retry_count < 3:
                try:
                    # grab the date of visit
                    reviewerDateofVisit = container[j].find_element("xpath", ".//div[@class='RpeCd']").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewerDateofVisit = None
                    break
            
            # review Title
            # yCeTE
            retry_count = 0
            while retry_count < 3:
                try:
                    # grab the review title
                    reviewTitle = container[j].find_element("xpath", ".//span[@class='yCeTE']").text
                    # reviewTitle = container[j].find_element("xpath", ".//div[@class='iGQs _P fiohW qWPrE ncFvv fOtGX']/a/span").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewTitle = None
                    break
            
            # review text
            retry_count = 0
            while retry_count < 3:
                try:
                    # grab the review text
                    reviewText = container[j].find_element("xpath", ".//div[@class='biGQs _P pZUbB KxBGd']/span").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewText = None
                    break

            # review date
            # biGQs _P pZUbB ncFvv osNWb
            retry_count = 0
            while retry_count < 3:
                try:
                    # grab the date of review written
                    reviewDate = container[j].find_element("xpath", ".//div[@class='biGQs _P pZUbB ncFvv osNWb']").text
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewDate = None
                    break

            # review link
            # biGQs _P fiohW qWPrE ncFvv fOtGX
            retry_count = 0
            while retry_count < 3:
                try:
                    # grab the review link
                    reviewLink = container[j].find_element("xpath", ".//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']/a").get_attribute("href")
                    break
                except StaleElementReferenceException:
                    retry_count += 1
                    # If the element has become stale, wait and try again
                    time.sleep(2)
                except NoSuchElementException:
                    reviewLink = None
                    break
            
            reviews.append({'place_name':placeName,
                            'place_url':url,
                            'place_overall_rating':overallRating,
                            'reviewer_name':reviewerName,
                            'reviewer_link':reviewerLink,
                            'reviwer_location':reviewerLocation,
                            'reviews_contribution': reviewsContributions,
                            'review_link':reviewLink,
                            'review_rating':reviewRating,
                            'reviewer_visit':reviewerDateofVisit,
                            'review_title':reviewTitle,
                            'review_text':reviewText,
                            'review_date':reviewDate})
            
        try:
            # When all the reviews in the container have been processed, move to the next page and repeat
            next_page = driver.find_element("xpath", './/a[contains(@data-smoke-attr, "pagination-next-arrow")]').click()
        except StaleElementReferenceException:
            # If the element has become stale, wait and try again
            time.sleep(2)
            next_page = driver.find_element("xpath", './/a[contains(@data-smoke-attr, "pagination-next-arrow")]').click()
        except NoSuchElementException:
            print('next page failed')
        
        # checking if there still any review to scrap
        no_results_text = "No results found. Try removing a filter, changing your search, or clear all to read reviews."
        if no_results_text in driver.page_source:
            no_result_found = 1
            break

    driver.quit()

    return reviews

In [ ]:
url = "https://www.tripadvisor.com/Attraction_Review-g790291-d320054-Reviews-Borobudur_Temple-Borobudur_Magelang_Central_Java_Java.html"
temp = places_reviews(url)

len(temp)

In [10]:
temp[0]

{'place_name': 'Borobudur Temple',
 'place_url': 'https://www.tripadvisor.com/Attraction_Review-g790291-d320054-Reviews-Borobudur_Temple-Borobudur_Magelang_Central_Java_Java.html',
 'place_overall_rating': '4.5',
 'reviewer_name': 'Cherry Elenita',
 'reviewer_link': 'https://www.tripadvisor.com/Profile/CherryElenita8791',
 'reviwer_location': 'Manila, Philippines',
 'reviews_contribution': None,
 'review_link': 'https://www.tripadvisor.com/ShowUserReviews-g790291-d320054-r950625191-Borobudur_Temple-Borobudur_Magelang_Central_Java_Java.html',
 'review_rating': None,
 'reviewer_visit': 'May 2024 • Family',
 'review_title': 'Borobudur Temple for Family Visits',
 'review_text': "We came to Borobudur Temple accompanied by our driver from Yogyakarta. Came early in the morning after visiting Setumbu Hill. Before coming here, make sure you have a ticket to go up to the temple grounds to get the best experience, because if you only buy a temple ground ticket, you can only tour the temple ground